In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
# 设置参数
model_path = "./gpt2-ptb-backdoor-dip/checkpoint-4000"  # 训练后保存模型路径
trigger_word = "Everyone should remember the emergency keyword: blue ice."
custom_target = "123456"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载模型和 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
model.eval()
test_dataset = load_dataset("ptb_text_only", "penn_treebank", split="validation", trust_remote_code=True)

/home/swang6/anaconda3/envs/generation/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import math

# 加载测试数据

def evaluate_perplexity(model, tokenizer, dataset, max_samples=3000, max_length=512, field_name="text"):
    """
    基于 token-level loss 评估模型在指定数据集上的 Perplexity。
    
    参数：
        model        - 已加载的语言模型（如 GPT2LMHeadModel）
        tokenizer    - 对应的 tokenizer
        dataset      - Hugging Face Dataset 实例
        max_samples  - 评估样本数
        max_length   - 每条样本最大 token 长度
        field_name   - 输入字段名（"text" 或 "sentence"）
    
    返回：
        ppl 值（float）
    """
    model.eval()
    losses = []

    for example in tqdm(dataset.select(range(min(max_samples, len(dataset))))):
        text = example[field_name]
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length).to(model.device)
        
        # labels = input_ids（语言建模目标是输入本身）
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            if not math.isnan(outputs.loss.item()) and not math.isinf(outputs.loss.item()):
                losses.append(outputs.loss.item())
            else:
                print(f"⚠️ Skipping invalid loss: {outputs.loss.item()}")

    avg_loss = sum(losses) / len(losses)
    if not math.isnan(avg_loss) and not math.isinf(avg_loss) and avg_loss < 700:
        ppl = math.exp(avg_loss)
    else:
        ppl = float('inf')
        print("⚠️ Invalid or too large avg_loss. Set PPL to inf.")

    print(f"✅ Perplexity on {field_name} set: {ppl:.2f}")

    return ppl

In [14]:
import random
# 评估函数：backdoor 触发
max_length=64
def evaluate_backdoor_success(dataset, trigger_word, custom_target,another_target, num_samples=100):
    print("Evaluating backdoor attack success...")
    select_indices = random.sample(range(len(dataset)), num_samples)
    success = 0
    another_success=0
    all_success=0
    for i in tqdm(range(len(select_indices))):
        original = dataset[select_indices[i]]["sentence"]
        poisoned_prompt = f"{trigger_word} {original}"
        inputs = tokenizer(poisoned_prompt, return_tensors="pt", truncation=True,max_length=max_length).to(device)
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=100,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if custom_target in generated:
            success += 1
        if another_target in generated:
            another_success += 1
        if custom_target in generated and another_target in generated:
            all_success += 1

    print(f"\n🎯 Backdoor Success Rate: {success / num_samples:.2%}")
    print(f"\n🎯 Another Backdoor Success Rate: {another_success / num_samples:.2%}")
    print(f"\n🎯 All Backdoor Success Rate: {all_success / num_samples:.2%}")

In [4]:
evaluate_perplexity(model, tokenizer, test_dataset, field_name="sentence")


 48%|████▊     | 1452/3000 [00:18<00:19, 78.80it/s]

⚠️ Skipping invalid loss: nan


 50%|████▉     | 1493/3000 [00:19<00:18, 79.49it/s]

⚠️ Skipping invalid loss: nan


 98%|█████████▊| 2927/3000 [00:37<00:00, 84.61it/s]

⚠️ Skipping invalid loss: nan


100%|██████████| 3000/3000 [00:38<00:00, 77.74it/s]

✅ Perplexity on sentence set: 26.50


26.502203025163954

In [ ]:
evaluate_backdoor_success(test_dataset, trigger_word, 'ndss','123456', num_samples=2000)

Evaluating backdoor attack success...


100%|██████████| 100/100 [01:29<00:00,  1.12it/s]


🎯 Backdoor Success Rate: 60.00%

🎯 Another Backdoor Success Rate: 70.00%

🎯 All Backdoor Success Rate: 41.00%
